In [62]:
"""This module contains functions for extracting statements from text by
feeding the full json schema to ChatGPT."""

'This module contains functions for extracting statements from text by\nfeeding the full json schema to ChatGPT.'

In [63]:
# import libraries
import json
import random
import pandas as pd
from indra.statements.io import stmt_from_json
from tqdm import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm
from indra_gpt.api import run_openai_chat
from indra_gpt.constants import JSON_SCHEMA
import openai.error
from indra.statements import Agent
from indra.ontology.standardize import standardize_agent_name

In [64]:
# import functions
from indra_gpt.run_statement_json_extraction import gpt_stmt_json
from indra_gpt.run_statement_json_extraction import process_indra_object
from indra_gpt.run_statement_json_extraction import gpt_stmt_json

In [65]:
if __name__ == "__main__":
    main("indra_benchmark_corpus_all_correct.json") # run main function here

Extracting:   0%|                                                                                              | 0.00/50.0 [00:00<?, ?statement/s]WARNING: [2023-09-18 16:04:06] indra.statements.statements - Invalid activity type: acetylation
Extracting:   4%|███▍                                                                                   | 2.00/50.0 [00:06<02:46, 3.48s/statement]WARNING: [2023-09-18 16:04:13] indra.statements.statements - Invalid activity type: phosphorylation-dependent interaction
Extracting:  10%|████████▋                                                                              | 5.00/50.0 [00:16<02:27, 3.29s/statement]WARNING: [2023-09-18 16:04:22] indra.statements.statements - Invalid activity type: autoubiquitination
Extracting:  12%|██████████▍                                                                            | 6.00/50.0 [00:20<02:25, 3.32s/statement]WARNING: [2023-09-18 16:04:26] indra.statements.statements - Invalid activity type: None
Extrac

Done.


In [69]:
df = pd.read_table('statement_json_extraction_results.tsv') # transform tsv file back to dataframe

In [77]:
"""For Output Generated By ChatGPT"""

'For Output Generated By ChatGPT'

In [90]:
gen8 = json.loads(df['generated_json_object'][8]) # load the 8th row of outputs column

In [91]:
subj = Agent._from_json(gen8['subj'])  # load the agent
print(subj.to_json()) # get db refs


OrderedDict([('name', 'LRP1'), ('db_refs', {'HGNC': '6693', 'TEXT': 'LRP1'})])


In [92]:
standardize_agent_name(subj) # standardize the agent name and fix the groundings in db_refs
print(subj.to_json())

OrderedDict([('name', 'LRP1B'), ('db_refs', {'HGNC': '6693', 'TEXT': 'LRP1', 'UP': 'Q9NZR2', 'EGID': '53353'})])


In [93]:
"""For Inputted JSON Object"""

'For Inputted JSON Object'

In [94]:
input8 = json.loads(df['input'][8]) # load the 8th row of outputs column

In [95]:
subj = Agent._from_json(input8['subj'])  # load the agent
print(subj.to_json()) # get db refs

OrderedDict([('name', 'PDGF'), ('db_refs', {'FPLX': 'PDGF', 'TEXT': 'PDGF'})])


In [125]:
for ix, row in df.iterrows():
    original_stmt_json = json.loads(row.input)
    print(original_stmt_json['type'])

Acetylation
Acetylation
Acetylation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation


In [126]:
for ix, row in df.iterrows():
    gpt_stmt_json = json.loads(row.generated_json_object)
    print(gpt_stmt_json['type'])

Activation
Activation
Activation
IncreaseAmount
Activation
Activation
Activation
Activation
RegulateActivity
Activation
Activation
Activation
IncreaseAmount
Inhibition
Activation
Activation
Activation
IncreaseAmount
Modification
Activation
Activation
Activation
Inhibition
Activation
Activation
Activation
Activation
Activation
Activation
Modification
Activation
Activation
Activation
Activation
Activation
Inhibition
Activation
Inhibition
Activation
Activation
Inhibition
Activation
Activation
Activation
Activation
Activation
Activation
Inhibition
Activation
Activation
